In [1]:
#load all the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor 
from sklearn import decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

LOAD TRAINING DATA

In [2]:
#upload training data
from google.colab import files
uploaded=files.upload()

Saving R_calculated_v2_11_Day_Interval_train.csv to R_calculated_v2_11_Day_Interval_train.csv


In [3]:
#load data to dataframe
import io
df=pd.read_csv(io.StringIO(uploaded['R_calculated_v2_11_Day_Interval_train.csv'].decode('utf-8')),sep=",")
df.head()

,Mean(R),retail_and_recreation_percent_change_from_baseline_11,grocery_and_pharmacy_percent_change_from_baseline_11,parks_percent_change_from_baseline_11,transit_stations_percent_change_from_baseline_11,workplaces_percent_change_from_baseline_11,residential_percent_change_from_baseline_11,retail_and_recreation_percent_change_from_baseline_10,grocery_and_pharmacy_percent_change_from_baseline_10,parks_percent_change_from_baseline_10,transit_stations_percent_change_from_baseline_10,workplaces_percent_change_from_baseline_10,residential_percent_change_from_baseline_10,retail_and_recreation_percent_change_from_baseline_9,grocery_and_pharmacy_percent_change_from_baseline_9,parks_percent_change_from_baseline_9,transit_stations_percent_change_from_baseline_9,workplaces_percent_change_from_baseline_9,residential_percent_change_from_baseline_9,retail_and_recreation_percent_change_from_baseline_8,grocery_and_pharmacy_percent_change_from_baseline_8,parks_percent_change_from_baseline_8,transit_stations_percent_change_from_baseline_8,workplaces_percent_change_from_baseline_8,residential_percent_change_from_baseline_8,retail_and_recreation_percent_change_from_baseline_7,grocery_and_pharmacy_percent_change_from_baseline_7,parks_percent_change_from_baseline_7,transit_stations_percent_change_from_baseline_7,workplaces_percent_change_from_baseline_7,residential_percent_change_from_baseline_7,retail_and_recreation_percent_change_from_baseline_6,grocery_and_pharmacy_percent_change_from_baseline_6,parks_percent_change_from_baseline_6,transit_stations_percent_change_from_baseline_6,workplaces_percent_change_from_baseline_6,residential_percent_change_from_baseline_6,retail_and_recreation_percent_change_from_baseline_5,grocery_and_pharmacy_percent_change_from_baseline_5,parks_percent_change_from_baseline_5,transit_stations_percent_change_from_baseline_5,workplaces_percent_change_from_baseline_5,residential_percent_change_from_baseline_5,retail_and_recreation_percent_change_from_baseline_4,grocery_and_pharmacy_percent_change_from_baseline_4,parks_percent_change_from_baseline_4,transit_stations_percent_change_from_baseline_4,workplaces_percent_change_from_baseline_4,residential_percent_change_from_baseline_4,retail_and_recreation_percent_change_from_baseline_3,grocery_and_pharmacy_percent_change_from_baseline_3,parks_percent_change_from_baseline_3,transit_stations_percent_change_from_baseline_3,workplaces_percent_change_from_baseline_3,residential_percent_change_from_baseline_3,retail_and_recreation_percent_change_from_baseline_2,grocery_and_pharmacy_percent_change_from_baseline_2,parks_percent_change_from_baseline_2,transit_stations_percent_change_from_baseline_2,workplaces_percent_change_from_baseline_2,residential_percent_change_from_baseline_2,retail_and_recreation_percent_change_from_baseline_1,grocery_and_pharmacy_percent_change_from_baseline_1,parks_percent_change_from_baseline_1,transit_stations_percent_change_from_baseline_1,workplaces_percent_change_from_baseline_1,residential_percent_change_from_baseline_1
0,1.563620,5,0,15,1,0,0,6,1,25,0,1,-1,5,-2,9,-1,-19,5,0,-1,6,3,2,0,3,0,14,3,2,0,2,0,2,2,2,0,2,-2,19,6,3,0,9,4,68,8,3,-2,7,3,77,6,3,-1,2,-1,0,2,3,1,4,0,2,4,2,0
1,1.680111,6,1,25,0,1,-1,5,-2,9,-1,-19,5,0,-1,6,3,2,0,3,0,14,3,2,0,2,0,2,2,2,0,2,-2,19,6,3,0,9,4,68,8,3,-2,7,3,77,6,3,-1,2,-1,0,2,3,1,4,0,2,4,2,0,6,2,-3,3,1,0
2,1.838417,5,-2,9,-1,-19,5,0,-1,6,3,2,0,3,0,14,3,2,0,2,0,2,2,2,0,2,-2,19,6,3,0,9,4,68,8,3,-2,7,3,77,6,3,-1,2,-1,0,2,3,1,4,0,2,4,2,0,6,2,-3,3,1,0,2,0,-6,2,-3,2
3,1.821975,0,-1,6,3,2,0,3,0,14,3,2,0,2,0,2,2,2,0,2,-2,19,6,3,0,9,4,68,8,3,-2,7,3,77,6,3,-1,2,-1,0,2,3,1,4,0,2,4,2,0,6,2,-3,3,1,0,2,0,-6,2,-3,2,4,1,7,7,1,0
4,1.662027,3,0,14,3,2,0,2,0,2,2,2,0,2,-2,19,6,3,0,9,4,68,8,3,-2,7,3,77,6,3,-1,2,-1,0,2,3,1,4,0,2,4,2,0,6,2,-3,3,1,0,2,0,-6,2,-3,2,4,1,7,7,1,0,11,5,13,7,2,-2


# Tuned RF and PCA

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# split the independent and response variables
X4 =df.drop('Mean(R)', axis = 1)
y4 = df['Mean(R)']   
from sklearn.model_selection import train_test_split 
  
#performing train/test split
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size = 0.33, random_state = 0) 
# performing preprocessing part 
from sklearn.preprocessing import StandardScaler 
sc = StandardScaler() 
  
X4_train = sc.fit_transform(X4_train) 
X4_test = sc.transform(X4_test) 

pca2 = PCA(n_components=5) 
  
X4_train = pca2.fit_transform(X4_train) 
X4_test = pca2.transform(X4_test)
#parameters obtained from GridSearch

'''
'pca__n_components': 5,
 'regressor__bootstrap': True,
 'regressor__max_depth': 80,
 'regressor__max_features': 3,
 'regressor__min_samples_leaf': 3,
 'regressor__min_samples_split': 8,
 'regressor__n_estimators': 100}
'''


#Fitting best parameters to the model
regressor4 = RandomForestRegressor(n_estimators=100,min_samples_split=8,min_samples_leaf=3,max_features=3,max_depth=80, bootstrap='True') 
  
# fit the regressor with x and y data 
regressor4.fit(X4_train,y4_train)   

#mse with Tuned Parameters

y4_pred = regressor4.predict(X4_test)
from sklearn.metrics import mean_squared_error
meanerror = mean_squared_error(y4_test, y4_pred)
print("Training Error:")
print(meanerror)


Training Error:
0.007839170530652095


# Testing Data

In [5]:
#upload testing data
from google.colab import files
uploaded=files.upload()

Saving R_calculated_v2_11_Day_Interval_test.xlsx to R_calculated_v2_11_Day_Interval_test.xlsx


In [6]:
import io
test=pd.read_excel(io.BytesIO(uploaded['R_calculated_v2_11_Day_Interval_test.xlsx']))
test.head()

,Mean(R),retail_and_recreation_percent_change_from_baseline_11,grocery_and_pharmacy_percent_change_from_baseline_11,parks_percent_change_from_baseline_11,transit_stations_percent_change_from_baseline_11,workplaces_percent_change_from_baseline_11,residential_percent_change_from_baseline_11,retail_and_recreation_percent_change_from_baseline_10,grocery_and_pharmacy_percent_change_from_baseline_10,parks_percent_change_from_baseline_10,transit_stations_percent_change_from_baseline_10,workplaces_percent_change_from_baseline_10,residential_percent_change_from_baseline_10,retail_and_recreation_percent_change_from_baseline_9,grocery_and_pharmacy_percent_change_from_baseline_9,parks_percent_change_from_baseline_9,transit_stations_percent_change_from_baseline_9,workplaces_percent_change_from_baseline_9,residential_percent_change_from_baseline_9,retail_and_recreation_percent_change_from_baseline_8,grocery_and_pharmacy_percent_change_from_baseline_8,parks_percent_change_from_baseline_8,transit_stations_percent_change_from_baseline_8,workplaces_percent_change_from_baseline_8,residential_percent_change_from_baseline_8,retail_and_recreation_percent_change_from_baseline_7,grocery_and_pharmacy_percent_change_from_baseline_7,parks_percent_change_from_baseline_7,transit_stations_percent_change_from_baseline_7,workplaces_percent_change_from_baseline_7,residential_percent_change_from_baseline_7,retail_and_recreation_percent_change_from_baseline_6,grocery_and_pharmacy_percent_change_from_baseline_6,parks_percent_change_from_baseline_6,transit_stations_percent_change_from_baseline_6,workplaces_percent_change_from_baseline_6,residential_percent_change_from_baseline_6,retail_and_recreation_percent_change_from_baseline_5,grocery_and_pharmacy_percent_change_from_baseline_5,parks_percent_change_from_baseline_5,transit_stations_percent_change_from_baseline_5,workplaces_percent_change_from_baseline_5,residential_percent_change_from_baseline_5,retail_and_recreation_percent_change_from_baseline_4,grocery_and_pharmacy_percent_change_from_baseline_4,parks_percent_change_from_baseline_4,transit_stations_percent_change_from_baseline_4,workplaces_percent_change_from_baseline_4,residential_percent_change_from_baseline_4,retail_and_recreation_percent_change_from_baseline_3,grocery_and_pharmacy_percent_change_from_baseline_3,parks_percent_change_from_baseline_3,transit_stations_percent_change_from_baseline_3,workplaces_percent_change_from_baseline_3,residential_percent_change_from_baseline_3,retail_and_recreation_percent_change_from_baseline_2,grocery_and_pharmacy_percent_change_from_baseline_2,parks_percent_change_from_baseline_2,transit_stations_percent_change_from_baseline_2,workplaces_percent_change_from_baseline_2,residential_percent_change_from_baseline_2,retail_and_recreation_percent_change_from_baseline_1,grocery_and_pharmacy_percent_change_from_baseline_1,parks_percent_change_from_baseline_1,transit_stations_percent_change_from_baseline_1,workplaces_percent_change_from_baseline_1,residential_percent_change_from_baseline_1
0,0.922479,-5,6,139,-7,-33,9,-11,1,217,-3,-33,8,-15,2,264,5,-10,1,-11,-2,176,-2,-13,1,-5,2,142,-6,-34,8,-5,2,160,-6,-34,8,-5,1,115,-9,-33,9,-6,2,127,-8,-33,9,-11,-1,216,-2,-33,8,-14,2,285,6,-10,1,-9,0,252,2,-11,0
1,0.925508,-11,1,217,-3,-33,8,-15,2,264,5,-10,1,-11,-2,176,-2,-13,1,-5,2,142,-6,-34,8,-5,2,160,-6,-34,8,-5,1,115,-9,-33,9,-6,2,127,-8,-33,9,-11,-1,216,-2,-33,8,-14,2,285,6,-10,1,-9,0,252,2,-11,0,-9,-4,74,-10,-35,10
2,0.925521,-15,2,264,5,-10,1,-11,-2,176,-2,-13,1,-5,2,142,-6,-34,8,-5,2,160,-6,-34,8,-5,1,115,-9,-33,9,-6,2,127,-8,-33,9,-11,-1,216,-2,-33,8,-14,2,285,6,-10,1,-9,0,252,2,-11,0,-9,-4,74,-10,-35,10,-3,4,176,-5,-33,8
3,0.922887,-11,-2,176,-2,-13,1,-5,2,142,-6,-34,8,-5,2,160,-6,-34,8,-5,1,115,-9,-33,9,-6,2,127,-8,-33,9,-11,-1,216,-2,-33,8,-14,2,285,6,-10,1,-9,0,252,2,-11,0,-9,-4,74,-10,-35,10,-3,4,176,-5,-33,8,-3,3,164,-6,-34,8
4,0.924560,-5,2,142,-6,-34,8,-5,2,160,-6,-34,8,-5,1,115,-9,-33,9,-6,2,127,-8,-33,9,-11,-1,216,-2,-

In [7]:
#split data into independent and target variables
X =test.drop('Mean(R)', axis = 1)
y = test['Mean(R)']  
#scale data to perform PCA
X = sc.transform(X) 
X = pca2.transform(X) 

y_pred = regressor4.predict(X)
from sklearn.metrics import mean_squared_error
meanerror = mean_squared_error(y, y_pred)
print("Testing Error:")
print(meanerror)

Testing Error:
0.003874137934120683
